<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/OpenAI_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Function Call + LangChain Example

## [Youtube video covering this notebook](https://youtu.be/Sn9z6ey5zo4)
- Example 1 (Replicate the openai example
- Example 2 (Sum of two numbers) Step by Step breakdown.
- Example 3 (Web surfing (kinda chatgpt plugin))
- More examples link

# ⚙️ Setup

In [1]:
%%capture
!pip install langchain openai watermark

In [2]:
%load_ext watermark
%watermark -a "Sudarshan Koirala" -vmp langchain,openai

Author: Sudarshan Koirala

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

langchain: 0.0.200
openai   : 0.27.8

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.15.107+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [6]:
import os
import json
import openai
import warnings

warnings.filterwarnings("ignore")

In [4]:
# get your openai api key from https://platform.openai.com/account/api-keys 🔑
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

··········


# Examples

## Example 1 from OpenAI documentation

In [7]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "id": "chatcmpl-7Rca0O4WJ6fhY4vQkUVWZ4NPMDek5",
  "object": "chat.completion",
  "created": 1686817512,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston is sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 69,
    "completion_tokens": 17,
    "total_tokens": 86
  }
}



## Example 2 (Sum of two numbers)

In [70]:
# lets define a function
def get_addition(number1,number2):
    """Get the sum of two numbers """
    return str(number1+number2)

In [71]:
# functions: A list of functions the model may generate JSON inputs for.
# name: string; Required; The name of the function to be called.
# description: string; Optional; The description of what the function does.
# parameters; object; Optional; The parameters the functions accepts, described as a JSON Schema object.
functions = [
        {
            "name": "get_addition",
            "description": "Get the addition of two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                        "number1": {
                            "type": "integer",
                            "description": "The first number, e.g. 1",
                        },
                        "number2": {
                            "type": "integer",
                            "description": "The second number, e.g. 1",
                        },

                    },
                "required": ["number1", "number2"],
            },
        }
    ]

In [72]:
# Step 1: send model the user query and what functions it has access to
# functions: A list of functions the model may generate JSON inputs for.
# function_call: Controls how the model responds to function calls.
# "auto" means the model can pick between an end-user or calling a function.
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "What's the sum of 5 and 7?"}],
    functions=functions,
    function_call="auto",
)

message = response["choices"][0]["message"]

In [73]:
message

<OpenAIObject at 0x7fa0843e91c0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_addition",
    "arguments": "{\n  \"number1\": 5,\n  \"number2\": 7\n}"
  }
}

In [76]:
# Step 2, check if the model wants to call a function
if message.get("function_call"):
        function_name = message["function_call"]["name"]
        arguments = json.loads(message["function_call"]["arguments"])

        print(arguments)

        number1=arguments["number1"]
        number2=arguments["number2"]

function_name

{'number1': 5, 'number2': 7}


'get_addition'

In [77]:
# Step 3, call the function
function_response = get_addition(
            number1,
            number2,
        )

In [78]:
function_response

'12'

In [59]:
# Step 4, send model the info on the function call and function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What is the sum of 5 and 7"},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        },
    ],
)

In [60]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7Rdp1ONVOF6B9nYmpm9z4m2qfD4xf at 0x7fa08d50db70> JSON: {
  "id": "chatcmpl-7Rdp1ONVOF6B9nYmpm9z4m2qfD4xf",
  "object": "chat.completion",
  "created": 1686822287,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The sum of 5 and 7 is 12."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 52,
    "completion_tokens": 12,
    "total_tokens": 64
  }
}

In [61]:
# Step 4, send model the info on the function call and function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What is the sum of 10 and 7"},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        },
    ],
)

In [62]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7RdpRHJBApiv8Rcj81GgaUH7umZZ4 at 0x7fa08d3b39c0> JSON: {
  "id": "chatcmpl-7RdpRHJBApiv8Rcj81GgaUH7umZZ4",
  "object": "chat.completion",
  "created": 1686822313,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The sum of 10 and 7 is 17."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 52,
    "completion_tokens": 12,
    "total_tokens": 64
  }
}

## Example 3 (Web surfing (kinda chatgpt plugin))
- `gpt-3.5-turbo-0613` includes the same function calling as GPT-4

In [63]:
%%capture
!pip install duckpy

In [66]:
from duckpy import Client

In [65]:
duckduckgo_client = Client()

In [67]:
def search_api(input):
    output = duckduckgo_client.search(input)
    return str(output)

In [69]:
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "Who is the winner of Champions League 2023?"}],
        functions=[
            {
                "name": "search_api",
                "description": "Search information from online",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "input": {
                            "type": "string",
                            "description": "Input query",
                        },
                    },
                    "required": ["input"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]
    print(message)

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]


        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = search_api(
            input = eval(message["function_call"]["arguments"]).get('input')
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "Who is the winner of Champions League 2023?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "search_api",
    "arguments": "{\n  \"input\": \"winner of Champions League 2023\"\n}"
  }
}
{
  "id": "chatcmpl-7Re1VmAQZJDRfLxJL6aDLFagLWODg",
  "object": "chat.completion",
  "created": 1686823061,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The winner of the UEFA Champions League in 2023 was Manchester City. They defeated Inter Milan 1-0 in the final, which took place at Istanbul's Atat\u00fcrk Olympic Stadium on June 10, 2023."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 2525,
    "completion_tokens": 48,
    "total_tokens": 2573
  }
}


## More Examples
1. [How to use functions with a knowledge base](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb)
2. [How to call functions with chat models](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb)
3. [LangChain Tools as OpenAI functions](https://python.langchain.com/en/latest/modules/agents/tools/tools_as_openai_functions.html)
4. [Langchain OpenAI Function Agent](https://github.com/hwchase17/langchain/blob/master/docs/modules/agents/agents/examples/openai_functions_agent.ipynb)